# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [40]:
# Write your code below.
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [41]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [42]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

#setting lags, returns, and hi_lo_range
import numpy as np
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
   hi_lo_range = lambda x: x['High'] - x['Low']
))

c:\ProgramData\anaconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
C:\Users\hgior\AppData\Local\Temp\ipykernel_11176\2886989308.py:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [46]:
# Write your code below.
import pandas as pd

#dd to df
df_feat = dd_feat.compute()

#rolling average return calculation 
df_feat = (df_feat.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg_return = x['returns'].rolling(window=10).mean())
))

In [45]:
df_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,,
HUM,2003-01-02,10.100000,10.350000,10.030000,10.310000,9.205755,856400,Health Care,Managed Health Care,2003,NaN,NaN,NaN,0.320001,NaN
HUM,2003-01-03,10.320000,10.480000,10.310000,10.400000,9.286114,795300,Health Care,Managed Health Care,2003,10.310000,9.205755,0.008729,0.169999,NaN
HUM,2003-01-06,10.480000,10.500000,10.260000,10.320000,9.214681,962000,Health Care,Managed Health Care,2003,10.400000,9.286114,-0.007692,0.240000,NaN
HUM,2003-01-07,10.260000,10.400000,10.250000,10.310000,9.205755,980000,Health Care,Managed Health Care,2003,10.320000,9.214681,-0.000969,0.150000,NaN
HUM,2003-01-08,10.320000,10.440000,10.250000,10.400000,9.286114,1071600,Health Care,Managed Health Care,2003,10.310000,9.205755,0.008729,0.190000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2021-12-27,34.230000,35.160000,34.220001,35.110001,32.587696,2857400,Information Technology,Communications Equipment,2021,34.130001,31.678104,0.028714,0.939999,0.003317
JNPR,2021-12-28,35.099998,35.549999,35.080002,35.430000,32.884724,3582300,Information Technology,Communications Equipment,2021,35.110001,32.587696,0.009115,0.469997,0.004199
JNPR,2021-12-29,35.450001,35.910000,35.349998,35.770000,33.200291,2243800,Information Technology,Communications Equipment,2021,35.430000,32.884724,0.009596,0.560001,0.006599


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return? No, it was not. It is possible to do this calculation using Dask.
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.